In [10]:
import tkinter as tk
import pygame
pygame.mixer.init()
from tkinter import messagebox
from PIL import Image, ImageTk, ImageFilter
import copy
import math
import random
import heapq
import os

# Constants 
BOARD_SIZE = 15
CELL = 50
CANVAS_W = BOARD_SIZE*CELL
CANVAS_H = BOARD_SIZE*CELL

MAX_DEPTH = 3
ANIM_STEPS = 6
ANIM_DELAY = 30
AI_DELAY = 200

NUM_CRYSTALS = 20
NUM_OBSTACLES = 30

# Assets paths
HUMAN_IMG_FILE = "human.png"
ROBOT_IMG_FILE = "robot.png"
ROBOT2_IMG_FILE = "robot2.png"       
CRYSTAL_IMG_FILE = "crystal.png"
OBSTACLE_IMG_FILE = "obstacle.png"
START_IMG_FILE = "Start.png"

START_SOUND= pygame.mixer.Sound("click.wav")
CRYSTAL_SOUND = pygame.mixer.Sound("crystal.wav")
WIN_SOUND = pygame.mixer.Sound("win.wav")
FAIR_SOUND = pygame.mixer.Sound("Fair.wav")

# ___________________________________________________ Main Game Class ___________________________________________________

class CrystalCapture:
    def __init__(self, root):
        self.root = root
        self.root.title(" Crystal Capture ")
        
        # game state
        self.state = {}
        self.ai_running = False 
        
        # assets
        self.ASSETS = {"human": None, "robot": None, "crystal": None, "obstacle": None, "robot2": None}
        self.CANVAS_IDS = {"player_H": None, "player_A": None, "crystals": {}, "obstacles": {}}  

        # setup GUI
        self.setup_gui()
        self.load_assets()
        self.reset_game()
    
# __________________________________________________ Helper Functions ____________________________________________________
    
    def in_bounds(self, pos):
        r,c = pos
        return 0<=r<BOARD_SIZE and 0<=c<BOARD_SIZE
    
    def manhattan(self,a,b):
        return abs(a[0]-b[0])+abs(a[1]-b[1])
    
    def get_moves(self,pos,obstacles,other_pos):
        r,c = pos
        candidates = [(r-1,c),(r+1,c),(r,c-1),(r,c+1)]
        moves = []
        for nr,nc in candidates:
            if 0<=nr<BOARD_SIZE and 0<=nc<BOARD_SIZE:
                if (nr,nc) in obstacles or (nr,nc)==other_pos:
                    continue
                moves.append((nr,nc))
        return moves 

#_________________________________________________ Pathfinding (A*) _______________________________________________________

    def Astar(self,start,goals,obstacles):
        open_set = [(0,start)]
        came_from = {}
        g_score = {start:0}
        f_score = {start:min(self.manhattan(start,g) for g in goals) if goals else 0}
        while open_set:
            _, current = heapq.heappop(open_set)
            if current in goals:
                path=[]
                while current in came_from:
                    path.append(current)
                    current = came_from[current]
                path.reverse()
                return path
            for neighbor in self.get_moves(current,obstacles,(-1,-1)):
                tentative_g = g_score[current]+1
                if neighbor not in g_score or tentative_g<g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + (min(self.manhattan(neighbor,g) for g in goals) if goals else 0)
                    heapq.heappush(open_set,(f_score[neighbor],neighbor))
        return []
#_____________________________________________________AI Evaluation______________________________________________________

    def is_terminal(self):
        return len(self.state["crystals"])==0

    def evaluate(self):
        # Score difference (AI score - Human score * SCORE WEIGHT)
        # SCORE WEIGHT = 20 => make score is more better than distance 
        val = self.state["A_score"] - self.state["H_score"]*20
        sum_a = sum(self.manhattan(self.state["A_pos"],c) for c in self.state["crystals"])
        sum_h = sum(self.manhattan(self.state["H_pos"],c) for c in self.state["crystals"])
        # DISTANCE WEIGHT = 1.3 
        if self.state["crystals"]:
            val += (sum_h - sum_a)*1.3
        return val

#____________________________________________________ State Transitions ____________________________________________________
    
 # Apply an action to the current state and return a new state. Action format: (player, new_position)
    
    def simulate_move(self, move, player):
        new_state = copy.deepcopy(self.state)
        if player=="H":
            new_state["H_pos"] = move
            if move in new_state["crystals"]:
                new_state["crystals"].remove(move)
                new_state["H_score"] += 1
                
        else:  # player == "A"
            new_state["A_pos"] = move
            if move in new_state["crystals"]:
                new_state["crystals"].remove(move)
                new_state["A_score"] +=1
                
        # Switch player
        new_state["player"] = "A" if player=="H" else "H"
        return new_state

#______________________________________________ Minimax with Alpha-Beta Pruning _______________________________________________

    def minimax_ab(self, state, depth, alpha, beta, maximizing):

        # Terminal check
        if depth==0 or self.is_terminal():
            return self.evaluate(), None
        player = "A" if maximizing else "H"
        pos = state["A_pos"] if maximizing else state["H_pos"]
        other = state["H_pos"] if maximizing else state["A_pos"]

        # Try A* path first if crystals exist
        if state["crystals"]:
            path = self.Astar(pos,state["crystals"],state["obstacles"])
            if path: return self.evaluate(), path[0]

        # Get all possible moves
        moves = self.get_moves(pos,state["obstacles"],other)
        if not moves: return self.evaluate(), None
            
        best_move = None
        
        if maximizing:
            max_val = -math.inf
            for mv in moves:
                new_state = self.simulate_move(mv,player)
                val,_ = self.minimax_ab(new_state,depth-1,alpha,beta,False)
                
                if val>max_val:
                    max_val = val
                    best_move = mv
                    
                alpha = max(alpha,val)
                if beta<=alpha:
                    break # Beta cutoff
                    
            return max_val,best_move
            
        else: # Minimizing player
            min_val = math.inf
            for mv in moves:
                new_state = self.simulate_move(mv,player)
                val,_ = self.minimax_ab(new_state,depth-1,alpha,beta,True)
                
                if val<min_val:
                    min_val=val
                    best_move=mv
                    
                beta = min(beta,val)
                if beta<=alpha:
                    break # Alpha cutoff
                    
            return min_val,best_move
        

#_____________________________________________________________ GUI ______________________________________________________________

    def setup_gui(self):
        # Create all GUI elements
        # Top frame with controls 
        self.top_frame = tk.Frame(self.root)
        self.top_frame.pack(pady=6, fill="x")

        # Mode selection
        self.mode_var = tk.StringVar(value="Human vs AI")
        tk.Label(self.top_frame,text="Mode:", font=("Segoe UI",11)).pack(side="left", padx=6)
        tk.Radiobutton(self.top_frame,text="Human vs AI",variable=self.mode_var,value="Human vs AI", command=self.reset_game).pack(side="left")
        tk.Radiobutton(self.top_frame,text="AI vs AI",variable=self.mode_var,value="AI vs AI",command=self.reset_game).pack(side="left")

        # Score labels
        self.human_score_label = tk.Label(self.top_frame,text="Human: 0", font=("Arial",12,"bold"), fg="#ff3b3b")
        self.human_score_label.pack(side="left", padx=12)
        self.ai_score_label = tk.Label(self.top_frame,text="AI: 0", font=("Arial",12,"bold"), fg="#4aa3ff")
        self.ai_score_label.pack(side="left", padx=6)
        self.turn_label = tk.Label(self.top_frame,text="Turn: Human", font=("Arial",12))
        self.turn_label.pack(side="left", padx=10)

        # Reset button
        self.reset_button = tk.Button(self.top_frame,text="Reset",command=self.reset_game)
        self.reset_button.pack(side="right", padx=10)

        # Canvas
        self.canvas = tk.Canvas(self.root,width=CANVAS_W,height=CANVAS_H,bg="#f7fbff")
        self.canvas.pack(padx=6,pady=6)
        self.canvas.bind("<Button-1>", self.click)
        self.mode_var.trace_add("write", lambda *args:self.on_mode_change())
        
    #Load all game images
    def load_image_resize(self, path, size):
        
        if not os.path.exists(path): 
            return None
        img = Image.open(path).convert("RGBA")
        img = img.resize(size, Image.LANCZOS)
        return ImageTk.PhotoImage(img)

    def load_assets(self):
        
        psize = (int(CELL*0.9), int(CELL*0.9))
        csize = (int(CELL*0.7), int(CELL*0.7))
        self.ASSETS["human"] = self.load_image_resize(HUMAN_IMG_FILE, psize)
        self.ASSETS["robot"] = self.load_image_resize(ROBOT_IMG_FILE, psize)
        self.ASSETS["crystal"] = self.load_image_resize(CRYSTAL_IMG_FILE, csize)
        self.ASSETS["obstacle"] = self.load_image_resize(OBSTACLE_IMG_FILE, csize)
        self.ASSETS["robot2"] = self.load_image_resize(ROBOT2_IMG_FILE, psize)

#____________________________________________________ Drawing _______________________________________________________________
    
    def draw_board(self,temp_pos=None,temp_player=None):

        # Draw entire game board
        self.canvas.delete("all")
        
        # Draw grid background
        for r in range(BOARD_SIZE):
            for c in range(BOARD_SIZE):
                x1,y1 = c*CELL,r*CELL
                x2,y2 = (c+1)*CELL,(r+1)*CELL
                self.canvas.create_rectangle(x1,y1,x2,y2,fill="#cab39c",outline="#ae8a67")
                
        # Draw obstacles
        self.CANVAS_IDS["obstacles"].clear()
        for ob in self.state["obstacles"]:
            r,c = ob
            x,y = c*CELL+CELL//2, r*CELL+CELL//2
            if self.ASSETS["obstacle"]:
                id_ = self.canvas.create_image(x,y,image=self.ASSETS["obstacle"])
            else:
                id_ = self.canvas.create_rectangle(c*CELL+6,r*CELL+6,(c+1)*CELL-6,(r+1)*CELL-6,fill="#333",outline="#111",width=2)
            self.CANVAS_IDS["obstacles"][ob]=id_
            
        # Draw crystals
        self.CANVAS_IDS["crystals"].clear()
        for cr in self.state["crystals"]:
            r,c = cr
            x,y = c*CELL+CELL//2, r*CELL+CELL//2
            if self.ASSETS["crystal"]:
                id_ = self.canvas.create_image(x,y,image=self.ASSETS["crystal"])
            else:
                id_ = self.canvas.create_oval(x-14,y-14,x+14,y+14,fill="#9cf3ff",outline="#35c5d6",width=2)
            self.CANVAS_IDS["crystals"][cr]=id_
            
        # Draw players
        mode = self.mode_var.get()
        if mode=="AI vs AI":
            img_H = self.ASSETS["robot2"]  
            img_A = self.ASSETS["robot"]
        else:
            img_H = self.ASSETS["human"]
            img_A = self.ASSETS["robot"]

        hr,hc = self.state["H_pos"]
        ar,ac = self.state["A_pos"]
        
        if temp_player=="H" and temp_pos: hr,hc=temp_pos
        if temp_player=="A" and temp_pos: ar,ac=temp_pos
            
        xH,yH = hc*CELL+CELL//2, hr*CELL+CELL//2
        xA,yA = ac*CELL+CELL//2, ar*CELL+CELL//2
        self.CANVAS_IDS["player_H"]=self.canvas.create_image(xH,yH,image=img_H) if img_H else self.canvas.create_oval(xH-18,yH-18,xH+18,yH+18,fill="#ff6b6b",outline="#d94141",width=2)
        self.CANVAS_IDS["player_A"]=self.canvas.create_image(xA,yA,image=img_A) if img_A else self.canvas.create_oval(xA-18,yA-18,xA+18,yA+18,fill="#4aa3ff",outline="#1f86ff",width=2)
        
        # Update score and turn labels
        mode = self.mode_var.get()
        if mode == "AI vs AI":
            self.human_score_label.config(text=f"Robot 1: {self.state['H_score']}")
            self.ai_score_label.config(text=f"Robot 2: {self.state['A_score']}")
            if self.state['player'] == "H":
                self.turn_label.config(text="Turn: Robot 1")
            else:
                self.turn_label.config(text="Turn: Robot 2")
        else:
            self.human_score_label.config(text=f"Human: {self.state['H_score']}")
            self.ai_score_label.config(text=f"AI: {self.state['A_score']}")
            if self.state['player'] == "H":
                self.turn_label.config(text="Turn: Human")
            else:
                self.turn_label.config(text="Turn: AI")

#_______________________________________________________ Game Flow ___________________________________________________________

    def reset_game(self):
        positions_used = set()
        def random_cell():
            while True:
                r=random.randint(0,BOARD_SIZE-1)
                c=random.randint(0,BOARD_SIZE-1)
                if (r,c) not in positions_used:
                    positions_used.add((r,c))
                    return (r,c)
        self.state = {
            "H_pos": random_cell(),
            "A_pos": random_cell(),
            "H_score":0,
            "A_score":0,
            "crystals":[random_cell() for _ in range(NUM_CRYSTALS)],
            "obstacles":[random_cell() for _ in range(NUM_OBSTACLES)],
            "player":"H"
        }
        self.ai_running=False
        self.game_active = True
        self.load_assets()
        self.draw_board()
        if self.mode_var.get()=="AI vs AI":
            self.root.after(AI_DELAY,self.ai_turn_step)

#________________________________________________ Apply a move and update game state ______________________________________________
            
    def apply_move(self,player,mv):
        if player=="H":
            self.state["H_pos"]=mv
            if mv in self.state["crystals"]:
                self.state["crystals"].remove(mv)
                self.state["H_score"]+=1
                CRYSTAL_SOUND.play()
        else:
            self.state["A_pos"]=mv
            if mv in self.state["crystals"]:
                self.state["crystals"].remove(mv)
                self.state["A_score"]+=1
                CRYSTAL_SOUND.play()
        self.state["player"]="A" if player=="H" else "H"
        self.draw_board()
        if len(self.state["crystals"]) > 0:
            reachable = False
            for cr in self.state["crystals"]:
                path_h = self.Astar(self.state["H_pos"], [cr], self.state["obstacles"])
                path_a = self.Astar(self.state["A_pos"], [cr], self.state["obstacles"])
                if path_h or path_a:
                    reachable = True
                    break

            # Check for unreachable crystals
            if not reachable:
                self.show_unreachable_dialog()
                return
        # Check if game is over
        if self.is_terminal():
            self.show_winner_dialog()

#________________________________________________________________________________________________________________________

    def animate_move(self,player,start,end,step=0):
        if step>=ANIM_STEPS:
            self.apply_move(player,end)
            self.ai_running=False
            return
        r0,c0 = start
        r1,c1 = end
        r = r0 + (r1-r0)*(step+1)/ANIM_STEPS
        c = c0 + (c1-c0)*(step+1)/ANIM_STEPS
        self.draw_board((r,c),player)
        self.ai_running=True
        self.root.after(ANIM_DELAY,self.animate_move,player,start,end,step+1)

#_____________________________________________________ AI Move Selection ________________________________________________
    
    def ai_turn_step(self):
        
        if self.is_terminal() or self.ai_running or not self.game_active: return
        current = self.state["player"]
        if self.mode_var.get()=="Human vs AI" and current=="H": return
        _,mv=self.minimax_ab(copy.deepcopy(self.state),MAX_DEPTH,-1e9,1e9,current=="A")

         # Fallback: if move conflicts with other player, choose random valid move
        if mv:
            start_pos = self.state["H_pos"] if current == "H" else self.state["A_pos"]
            other_pos = self.state["A_pos"] if current == "H" else self.state["H_pos"]
        if mv == other_pos: 
            possible_moves = self.get_moves(start_pos, self.state["obstacles"], other_pos) 
            if possible_moves:
                mv = random.choice(possible_moves)
            else:
                mv=None
        if mv:
            self.animate_move(current, start_pos, mv)
        self.root.after(ANIM_STEPS*ANIM_DELAY + AI_DELAY, self.ai_turn_step) 

#________________________________________________________________________________________________________________________

    def click(self,event):
        if self.mode_var.get()=="AI vs AI" or self.state["player"]!="H" or not self.game_active: return
        r,c = event.y//CELL, event.x//CELL
        if (r,c) in self.get_moves(self.state["H_pos"],self.state["obstacles"],self.state["A_pos"]):
            self.apply_move("H",(r,c))
            self.root.after(AI_DELAY,self.ai_turn_step)

#__________________________________________________________________________________________________________________________
    
    def on_mode_change(self):
        self.draw_board()
        if self.mode_var.get()=="AI vs AI":
            self.root.after(AI_DELAY,self.ai_turn_step) 

#__________________________________________________________________________________________________________________________

    def get_player_name(self, player):
        mode = self.mode_var.get()
        if mode == "AI vs AI":
            return "Robot 1" if player == "H" else "Robot 2"
        else:
            return "Human" if player == "H" else "AI"

#__________________________________________________________________________________________________________________________

    def show_winner_dialog(self):
        self.game_active = False
        H,A=self.state["H_score"],self.state["A_score"]
        if H==A:
            winner_name = "Tie"
        elif A > H:
            winner_name = self.get_player_name("A")
        else:
            winner_name = self.get_player_name("H")
        winner_img = None
        
        mode = self.mode_var.get()
        if mode == "AI vs AI":
            if winner_name == "Robot 1":
                winner_img = self.ASSETS["robot2"]
            elif winner_name == "Robot 2":
                winner_img = self.ASSETS["robot"]
        else:
            if winner_name == "Human":
                winner_img = self.ASSETS["human"]
            elif winner_name == "AI":
                winner_img = self.ASSETS["robot"]

        win=tk.Toplevel(self.root)
        win.title("Game Over")

        if winner_name != "Tie":
            WIN_SOUND.play() 
        else:
            FAIR_SOUND.play()
            
        win.configure(bg="#fff")
        win.geometry("360x260")
        win.resizable(False,False)
        tk.Label(win,text="END!",font=("Segoe UI",16,"bold"),bg="#fff").pack(pady=(12,4))
        
        if winner_name=="Tie":
            tk.Label(win,text=f"It's a tie! — {H}:{A}",font=("Segoe UI",12),bg="#fff").pack(pady=6)
        else:
            tk.Label(win,text=f"{winner_name} wins — {H}:{A}",font=("Segoe UI",12),bg="#fff").pack(pady=6)
        if winner_img:
            tk.Label(win,image=winner_img,bg="#fff").pack(pady=6)
            
        btn_frame=tk.Frame(win,bg="#fff")
        btn_frame.pack(pady=8)
        tk.Button(btn_frame,text="Play Again",command=lambda:[win.destroy(),self.reset_game()]).pack(side="left",padx=8)
        tk.Button(btn_frame,text="Close",command=win.destroy).pack(side="left",padx=8)

#__________________________________________________________________________________________________________________________

    def show_unreachable_dialog(self):
        self.game_active = False
        crystals_left = len(self.state["crystals"])
        unreachable_text = f"The {'last crystal is' if crystals_left == 1 else f'remaining {crystals_left} crystals are'} unreachable!"
        H,A=self.state["H_score"],self.state["A_score"]
        mode = self.mode_var.get()
        
        if H > A:
            winner_name = self.get_player_name("H")
        elif A > H:
            winner_name = self.get_player_name("A")
        else:
            winner_name = "Tie"
        
        if mode == "AI vs AI":
            if winner_name == "Robot 1":
                winner_img = self.ASSETS["robot2"]
            elif winner_name == "Robot 2":
                winner_img = self.ASSETS["robot"]
            else:
                winner_img = self.ASSETS["robot"]
        else:
            if winner_name == "Human":
                winner_img = self.ASSETS["human"]
            else:
                winner_img = self.ASSETS["robot"]
        
        win=tk.Toplevel(self.root)
        win.title("Game Stuck")
        win.configure(bg="#fff")
        win.geometry("360x260")
        win.resizable(False,False)
        tk.Label(win,text="STUCK!",font=("Segoe UI",16,"bold"),bg="#fff").pack(pady=(12,4))
        tk.Label(win,text=unreachable_text,font=("Segoe UI",12),bg="#fff").pack(pady=6)
            
        if winner_name=="Tie":
            tk.Label(win,text=f"It's a tie! — {H}:{A}",font=("Segoe UI",12),bg="#fff").pack(pady=6)
        else:
            tk.Label(win,text=f"{winner_name} wins — {H}:{A}",font=("Segoe UI",12),bg="#fff").pack(pady=6)
        if winner_img:
            tk.Label(win,image=winner_img,bg="#fff").pack(pady=6)
            
        btn_frame=tk.Frame(win,bg="#fff")
        btn_frame.pack(pady=8)
        tk.Button(btn_frame,text="Play Again",command=lambda:[win.destroy(),self.reset_game()]).pack(side="left",padx=8)
        tk.Button(btn_frame,text="Close",command=win.destroy).pack(side="left",padx=8)

#_________________________________________________ Main Start Screen _________________________________________________________


if __name__ == "__main__":
    root = tk.Tk()
    root.title(" Crystal Capture ")
    root.geometry(f"{CANVAS_W+80}x{CANVAS_H+120}")
    root.resizable(False, False)
    
#____________________________________________________ Main Canvas _____________________________________________________________
    
    start_canvas = tk.Canvas(root, width=CANVAS_W+80, height=CANVAS_H+120, highlightthickness=0)
    start_canvas.pack(fill="both", expand=True)

# ________________________________________________Load Background Image _______________________________________________________
    
    if os.path.exists(START_IMG_FILE):
        bg_img = Image.open(START_IMG_FILE)
        bg_img = bg_img.resize((CANVAS_W+80, CANVAS_H+120), Image.LANCZOS)
        bg_photo = ImageTk.PhotoImage(bg_img)
        start_canvas.bg_photo = bg_photo
        start_canvas.create_image(0,0,image=bg_photo,anchor="nw")

#_______________________________________________________ Glow Effect ____________________________________________________________
    
    GLOW_STEPS = ["#F4B342", "#d9fbff", "#ffffff", "#F79A19"]
    start_canvas.glow_index = 0
    title = start_canvas.create_text(
        (CANVAS_W+80)//2,
        int((CANVAS_H+120)*0.2),
        text="💎 Crystal Capture 💎",
        fill=GLOW_STEPS[0],
        font=("Palatino Linotype", 35, "bold")
    )

    def animate_glow():
        start_canvas.itemconfig(title, fill=GLOW_STEPS[start_canvas.glow_index])
        start_canvas.glow_index = (start_canvas.glow_index + 1) % len(GLOW_STEPS)
        start_canvas.after(250, animate_glow)

    animate_glow()

#__________________________________________________________ START Button _________________________________________________________
    
    btn_x = (CANVAS_W+80)//2
    btn_y = int((CANVAS_H+120)*0.7)
    shadow = start_canvas.create_oval(btn_x-115, btn_y-25, btn_x+115, btn_y+25, fill="#F4B342", outline="")
    oval_btn = start_canvas.create_oval(btn_x-120, btn_y-30, btn_x+120, btn_y+30, outline="#F4B342", width=3)
    btn_text = start_canvas.create_text(btn_x, btn_y, text="START", fill="#ffffff", font=("Segoe UI", 18, "bold"))

    def start_game(event=None):
        start_canvas.destroy()
        CrystalCapture(root)
        START_SOUND.play()

    start_canvas.tag_bind(oval_btn, "<Button-1>", start_game)
    start_canvas.tag_bind(btn_text, "<Button-1>", start_game)

    def on_enter(event):
        start_canvas.itemconfig(oval_btn, width=4)

    def on_leave(event):
        start_canvas.itemconfig(oval_btn, width=3)

    start_canvas.tag_bind(oval_btn, "<Enter>", on_enter)
    start_canvas.tag_bind(btn_text, "<Enter>", on_enter)
    start_canvas.tag_bind(oval_btn, "<Leave>", on_leave)
    start_canvas.tag_bind(btn_text, "<Leave>", on_leave)
    root.mainloop()